In [1]:
from cmu_112_graphics import *
from tkinter import *
import random

# Seven "standard" pieces (tetrominoes)
iPiece = [
    [  True,  True,  True,  True ]
]

jPiece = [
    [  True, False, False ],
    [  True,  True,  True ]
]

lPiece = [
    [ False, False,  True ],
    [  True,  True,  True ]
]

oPiece = [
    [  True,  True ],
    [  True,  True ]
]

sPiece = [
    [ False,  True,  True ],
    [  True,  True, False ]
]

tPiece = [
    [ False,  True, False ],
    [  True,  True,  True ]
]

zPiece = [
    [  True,  True, False ],
    [ False,  True,  True ]
]



def gameDimensions():
    rows, cols, cellSize, margin = 15, 10, 20, 25
#     rows, cols, cellSize, margin = 15, 10, 12, 2
    return (rows, cols, cellSize, margin)

def playTetris():
    (rows, cols, cellSize, margin) = gameDimensions()
    width, height = cols * cellSize + margin * 2, rows * cellSize + margin * 2
    runApp(width=width, height=height)
#     runApp(width=width, height=height, x=0, y=500)

def drawBoard(app, canvas):
    # draw cells
    for i in range(app.rows):
        for j in range(app.cols):
            drawCell(app, canvas, i, j, app.board[i][j])

def drawCell(app, canvas, row, col, fill):
    left = col * app.cellSize + app.margin
    top = row * app.cellSize + app.margin
    right = left + app.cellSize
    bottom = top + app.cellSize
    canvas.create_rectangle(left, top, right, bottom, fill=fill, width=3)

def appStarted(app):
    app.timerDelay = 200
    app.isGameOver = False
    app.score = 0
    
    (app.rows, app.cols, app.cellSize, app.margin) = gameDimensions()
    # init cells
    app.emptyColor = "blue"
    app.board = [[app.emptyColor] * app.cols for i in range(app.rows)]
    
    app.tetrisPieces = [iPiece, jPiece, lPiece, oPiece, sPiece, tPiece, zPiece]
    app.tetrisPieceColors = ["red", "yellow", "magenta", "pink", "cyan", "green", "orange"]
    
    newFallingPiece(app)
    
    # board test
#     app.board[0][0] = "red" # top-left is red
#     app.board[0][app.cols-1] = "white" # top-right is white
#     app.board[app.rows-1][0] = "green" # bottom-left is green
#     app.board[app.rows-1][app.cols-1] = "gray" # bottom-right is gray

def redrawAll(app, canvas):
    # draw background
    canvas.create_rectangle(0, 0, app.width, app.height, fill="orange", width=0)
    
    # draw board
    drawBoard(app, canvas)
    
    # draw score
    drawScore(app, canvas)
    
    # draw falling piece
    drawFallingPiece(app, canvas)
    
    # draw game over
    drawGameOver(app, canvas)

def newFallingPiece(app):
    app.randomIndex = random.randint(0, len(app.tetrisPieces) - 1)
    app.fallingPiece = app.tetrisPieces[app.randomIndex]
    
    app.fallingPieceRowBlocks, app.fallingPieceColBlocks = len(app.fallingPiece), len(app.fallingPiece[0])
    app.fallingPieceRowIdx, app.fallingPieceColIdx = 0, app.cols // 2 - app.fallingPieceColBlocks // 2
    
def drawFallingPiece(app, canvas):
    for i in range(app.fallingPieceRowBlocks):
        for j in range(app.fallingPieceColBlocks):
            if app.fallingPiece[i][j]:
                drawCell(app, canvas, app.fallingPieceRowIdx + i, app.fallingPieceColIdx + j, app.tetrisPieceColors[app.randomIndex])

def timerFired(app):
    if not app.isGameOver:
        if not moveFallingPiece(app, +1, 0):
            placeFallingPiece(app)
            removeFullRows(app)
            newFallingPiece(app)
            if not fallingPieceIsLegal(app):
                # game over
                app.isGameOver = True

def keyPressed(app, event):
    if not app.isGameOver:
        if event.key == 'Down':
            moveFallingPiece(app, +1, 0)
        elif event.key == 'Left':
            moveFallingPiece(app, 0, -1)
        elif (event.key == 'Right'):
            moveFallingPiece(app, 0, +1)
        elif (event.key == 'Up'):
            rotateFallingPiece(app)
    
    if (event.key == 'r'):
        appStarted(app)

def moveFallingPiece(app, drow, dcol):
    app.fallingPieceRowIdx += drow
    app.fallingPieceColIdx += dcol
    
    if not fallingPieceIsLegal(app):
        app.fallingPieceRowIdx -= drow
        app.fallingPieceColIdx -= dcol
        return False
    
    return True
    
def fallingPieceIsLegal(app):
    if app.fallingPieceRowIdx < 0 or app.fallingPieceRowIdx + app.fallingPieceRowBlocks > app.rows:
        return False
    if app.fallingPieceColIdx < 0 or app.fallingPieceColIdx + app.fallingPieceColBlocks > app.cols:
        return False
    
    for i in range(app.fallingPieceRowBlocks):
        for j in range(app.fallingPieceColBlocks):
            if app.fallingPiece[i][j] and app.board[app.fallingPieceRowIdx + i][app.fallingPieceColIdx + j] != app.emptyColor:
                return False
    return True

def rotateFallingPiece(app, mode="counterclockwise"):
    # backup falling piece info
    fallingPiece = app.fallingPiece
    fallingPieceRowIdx, fallingPieceColIdx = app.fallingPieceRowIdx, app.fallingPieceColIdx
    fallingPieceRowBlocks, fallingPieceColBlocks = app.fallingPieceRowBlocks, app.fallingPieceColBlocks
    
    # transpose
    app.fallingPiece = [[row[i] for row in app.fallingPiece] for i in range(app.fallingPieceColBlocks)]
    # upside down
    if mode == "counterclockwise":
        app.fallingPiece = [row for row in app.fallingPiece[::-1]]
    
    # update falling piece block size
    app.fallingPieceRowBlocks, app.fallingPieceColBlocks = len(app.fallingPiece), len(app.fallingPiece[0])
    
    # update falling piece position
    app.fallingPieceRowIdx = app.fallingPieceRowIdx + fallingPieceRowBlocks // 2 - app.fallingPieceRowBlocks // 2
    app.fallingPieceColIdx = app.fallingPieceColIdx + fallingPieceColBlocks // 2 - app.fallingPieceColBlocks // 2
    
    # restore if check fails
    if not fallingPieceIsLegal(app):
        app.fallingPiece = fallingPiece
        app.fallingPieceRowIdx, app.fallingPieceColIdx = fallingPieceRowIdx, fallingPieceColIdx
        app.fallingPieceRowBlocks, app.fallingPieceColBlocks = fallingPieceRowBlocks, fallingPieceColBlocks
    
def placeFallingPiece(app):
    for i in range(app.fallingPieceRowBlocks):
        for j in range(app.fallingPieceColBlocks):
            if app.fallingPiece[i][j]:
                app.board[app.fallingPieceRowIdx + i][app.fallingPieceColIdx + j] = app.tetrisPieceColors[app.randomIndex]

def drawGameOver(app, canvas):
    if app.isGameOver:
        canvas.create_rectangle(app.margin, app.margin + app.cellSize * 2, app.margin + app.cellSize * app.cols, app.margin + app.cellSize * 4, fill='black')
        canvas.create_text(app.width / 2, app.margin + app.cellSize * 3, text='Game Over !', font='Arial %d bold' % app.cellSize, fill='yellow')

def removeFullRows(app):
    # fetch all lines without emptyColor
    remove_lines = [row for row in app.board if not app.emptyColor in row]
    # remove these lines from board
    [app.board.remove(line) for line in remove_lines]
    # generate emptyColor lines with the same shape of lines removed
    emptyColorLines = [[app.emptyColor] * app.cols for i in range(len(remove_lines))]
    # stack
    [emptyColorLines.append(l) for l in app.board]
    app.board = emptyColorLines
        
    # scoring
    app.score += len(remove_lines)

def drawScore(app, canvas):
    canvas.create_text(app.width / 2, app.margin / 2, text='Score: %d' % app.score, font='Arial %d bold' % (app.margin / 2), fill='blue')
    
playTetris()

*** Closing TopLevelApp.  Bye! ***

